# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [3]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns=['MEDV'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [4]:
from sklearn.model_selection import train_test_split

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.20)

In [77]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

Fit the model and apply the model to the make test set predictions

In [78]:
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Calculate the residuals and the mean squared error

In [79]:
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test
print(test_residuals.head())

from sklearn.metrics import mean_squared_error

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train MSE:',train_mse)
print('Test MSE:',test_mse)

          MEDV
130   3.586555
184  -8.345192
465  -2.261089
352   2.560821
405  10.181401
Train MSE: 20.63480499384617
Test MSE: 26.546130426406325


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [80]:
def kfolds(data, k):
    # Force data as pandas dataframe
    df = pd.DataFrame(data)
    size = len(df)
    parts = []
    
    for i in range(k):
        if size%k == 0:
            sample_size = int(size/k)
        else:
            if len(df)%(k-i) == 0:
                sample_size = int(len(df)/(k-i))
            else:
                sample_size = int(np.floor(size/k)+1)
        subset = df.sample(sample_size)
        parts.append(subset)
        df.drop(subset.index, inplace=True)    
    return parts

In [81]:
folds = kfolds(pd.concat([X,y], axis=1), 5)

In [82]:
folds[0].head()

,CHAS,RM,DIS,B,LSTAT,MEDV
38,0.0,5.966,0.516323,0.991250,-0.092394,24.7
439,0.0,5.627,0.200306,1.000000,1.264870,12.8
52,0.0,6.511,0.757192,1.000000,-1.177815,25.0
62,0.0,6.456,0.781854,1.000000,-0.773600,22.2
153,0.0,5.709,0.152741,0.659716,0.647032,19.4


### Apply it to the Boston Housing Data

In [83]:
# Make sure to concatenate the data again

In [84]:
# Data is stored as a dictionary of dataframes

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [85]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(folds) if i!=n])
    test = folds[n]
    # Fit a linear regression model
    X_train = train.drop(columns=['MEDV'])
    y_train = train['MEDV']
    X_test = test.drop(columns=['MEDV'])
    y_test = test['MEDV']
    linreg.fit(X_train, y_train)
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    #Evaluate Train and Test Errors
    train_mse = mean_squared_error(y_train, y_hat_train)
    test_mse = mean_squared_error(y_test, y_hat_test)
    #Add Errors to lists
    test_errs.append(test_mse)
    train_errs.append(train_mse)
d = {'Train MSE':train_errs,'Test MSE':test_errs}
pd.DataFrame(d)


,Train MSE,Test MSE
0,22.374665,19.536285
1,20.400178,27.921127
2,23.368615,15.316311
3,19.680900,30.243415
4,22.077724,20.976973


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [86]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

sk_results = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")
sk_results

array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [87]:
sk_results.mean()

-30.451881143540298

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!